In [14]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [15]:
import logging
from datetime import datetime
from pathlib import Path
import pandas as pd

pd.set_option("display.max_rows", 102)
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

import optuna

from src.utils import (
    load_MovieLens,
    train_test_val_split,
    seed_everything,
    split_test_df,
    train,
    predict,
    enrich_train_with_negatives,
)
from src.models import AutoInt
from src.metrics import reccomendation_report

## Constants

In [16]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7
MODEL_NAME = "AutoInt"

In [17]:
seed_everything(RANDOM_STATE)

## Data

In [18]:
# subsample 30% of data for tuning
df_users, df_movies, df_ratings = load_MovieLens(DATA_FOLDER, sample_frac=0.3)

### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 20% of test.

In [19]:
df_train, df_test, df_val = train_test_val_split(df_ratings, df_movies, RANDOM_STATE)
df_test = pd.concat([df_test, df_val], ignore_index=True)

Enriching test:   0%|          | 0/1449 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/363 [00:00<?, ?it/s]

In [20]:
num_users = df_users["userId"].nunique()
num_items = df_movies["movieId"].nunique()

In [21]:
seed_everything(RANDOM_STATE)
df_train = enrich_train_with_negatives(df_train, df_movies)

  0%|          | 0/297511 [00:00<?, ?it/s]

### Loading additional data

In [22]:
# cos_dist = computeCosineSimilarities(df_train, "userId", "movieId", df_movies["movieId"].nunique())

# with open("../../data/cos_dist.pt", "wb") as f:
#     torch.save(cos_dist, f)

with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

In [23]:
ord_user = OrdinalEncoder()
user_cat = pd.DataFrame(
    ord_user.fit_transform(df_users[["gender", "occupation"]]),
    columns=ord_user.feature_names_in_,
).assign(userId=df_users["userId"])

ord_movie = OrdinalEncoder()
movie_cat = pd.DataFrame(
    ord_movie.fit_transform(df_movies[["genre"]]), columns=ord_movie.feature_names_in_
).assign(movieId=df_movies["movieId"])

ss_user = StandardScaler()
user_num = pd.DataFrame(
    ss_user.fit_transform(df_users[["age"]]), columns=ss_user.feature_names_in_
).assign(userId=df_users["userId"])

df_train = (
    df_train[["userId", "movieId", "label"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
    .rename(columns={"label": "action"})
)
df_test = (
    df_test[["userId", "movieId", "action"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
)

## Tuning

In [24]:
def objective(trial):
    # Define range of values to be tested for the hyperparameters
    num_attn_blocks = trial.suggest_int("num_attn_blocks", 1, 10)
    n_mlp_layers = trial.suggest_int("n_mlp_layers", 3, 10)
    mlp_layers_dim = trial.suggest_discrete_uniform("mlp_layers_dim", 16, 512, 16)
    mlp_kwargs_dropout = trial.suggest_categorical("mlp_kwargs_dropout", [True, False])
    mlp_kwargs_batchnorm = trial.suggest_categorical(
        "mlp_kwargs_batchnorm", [True, False]
    )
    mlp_kwargs_dropout_rate = trial.suggest_float("mlp_kwargs_dropout_rate", 0.1, 0.9)

    lr = trial.suggest_float("lr", 1e-4, 5e-3)
    n_epochs = trial.suggest_discrete_uniform("n_epochs", 5, 20, 2)

    # Generate the model
    seed_everything(RANDOM_STATE)
    autoint = AutoInt(
        task="classification",
        target=["action"],
        learning_rate=lr,
        head="LinearHead",
        max_epochs=int(n_epochs),
        layers="-".join([str(mlp_layers_dim) for _ in range(n_mlp_layers)]),
        num_attn_blocks=num_attn_blocks,
        verbose=False,
    )

    autoint.fit(df_train)

    # Evaluate
    df_test["pred"] = autoint.predict(df_test)
    pred, target, pred_items = split_test_df(
        df_test, "userId", "movieId", "pred", "action"
    )
    k = 15
    hit_rate = reccomendation_report(
        pred, target, pred_items, cos_dist, popularity, k=k
    )[f"Hit rate @ {k}"]

    return hit_rate.item()

In [25]:
# set up logging
directory = Path(f"optuna/{MODEL_NAME}")
if not directory.exists():
    directory.mkdir()
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

logger = logging.getLogger("optuna")
logger.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
file_handler = logging.FileHandler(
    f"{directory.as_posix()}/optuna_logs_{current_time}.log"
)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [26]:
# launch tuning
study = optuna.create_study(direction="maximize")
study.optimize(objective, timeout=3600)

[I 2024-05-13 17:51:08,516] A new study created in memory with name: no-name-7c17a0d8-e644-47ff-9bbf-082f8222bac7
/var/folders/b2/yfqb1mj945ncbc82gjz4yr7h0000gn/T/ipykernel_11751/1028126850.py:5: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  mlp_layers_dim = trial.suggest_discrete_uniform("mlp_layers_dim", 16, 512, 16)
/var/folders/b2/yfqb1mj945ncbc82gjz4yr7h0000gn/T/ipykernel_11751/1028126850.py:13: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  n_epochs = trial.suggest_discrete_uniform("n_epochs", 5, 20, 2)
/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/optuna/distributions.py:685: UserWarning: The distr

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ AutoIntBackbone  │ 43.3 K │
│ 1 │ _embedding_layer │ Embedding2dLayer │ 96.5 K │
│ 2 │ _head            │ LinearHead       │    386 │
│ 3 │ loss             │ CrossEntropyLoss │      0 │
└───┴──────────────────┴──────────────────┴────────┘

Trainable params: 140 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 140 K                                                                                                
Total estimated model params size (MB): 0

Output()

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_lightn
ing/trainer/connectors/logger_connector/result.py:433: It is recommended to use `self.log('valid_loss', ..., 
sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.

...

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_lightn
ing/trainer/connectors/logger_connector/result.py:433: It is recommended to use `self.log('valid_loss', ..., 
sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_lightn
ing/trainer/connectors/logger_connector/result.py:433: It is recommended to use `self.log('valid_accuracy', ..., 
sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_lightn
ing/trainer/connectors/logger_connector/result.py:433: It is recommended to use `self.log('train_accuracy', ..., 
sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.

`Trainer.fit` stopped: `max_epochs=19` reached.


/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
[I 2024-05-13 18:54:22,601] Trial 10 finished with value: 0.6683223247528076 and parameters: {'num_attn_blocks': 1, 'n_mlp_layers': 6, 'mlp_layers_dim': 272.0, 'mlp_kwargs_dropout': False, 'mlp_kwargs_batchnorm': True, 'mlp_kwargs_dropout_rate': 0.6143091642308008, 'lr': 0.0012284355704123